In [14]:
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam

df=pd.read_csv('forestfires.csv')
df

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,0,0,0,large
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,0,0,0,large
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,0,0,0,0,0,0,0,0,0,large
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,0,0,0,0,0,0,0,0,0,small


In [4]:
df1=df.drop(['month','day'],axis=1)
df1

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00,1,...,0,0,0,0,1,0,0,0,0,small
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00,0,...,0,0,0,0,0,0,0,1,0,small
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00,0,...,0,0,0,0,0,0,0,1,0,small
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00,1,...,0,0,0,0,1,0,0,0,0,small
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00,0,...,0,0,0,0,1,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44,0,...,0,0,0,0,0,0,0,0,0,large
513,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29,0,...,0,0,0,0,0,0,0,0,0,large
514,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16,0,...,0,0,0,0,0,0,0,0,0,large
515,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00,0,...,0,0,0,0,0,0,0,0,0,small


In [8]:
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()

In [6]:
df1['size_category']=lb.fit_transform(df1['size_category'])

In [10]:
df2=df1.values
print(df2)
df2.shape

[[ 86.2  26.2  94.3 ...   0.    0.    1. ]
 [ 90.6  35.4 669.1 ...   1.    0.    1. ]
 [ 90.6  43.7 686.9 ...   1.    0.    1. ]
 ...
 [ 81.6  56.7 665.6 ...   0.    0.    0. ]
 [ 94.4 146.  614.7 ...   0.    0.    1. ]
 [ 79.5   3.  106.7 ...   0.    0.    1. ]]


(517, 29)

In [11]:
x=df2[:,0:28]
y=df2[:,-1]
x.shape

(517, 28)

In [15]:
model = Sequential()
model.add(Dense(12, input_dim=28, activation='relu'))
model.add(Dense(28, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [16]:
model.fit(x, y, validation_split=0.33,epochs=100, batch_size=5)

Epoch 1/100
70/70 [==============================] - 4s 9ms/step - loss: 5.0394 - accuracy: 0.6358 - val_loss: 3.9273 - val_accuracy: 0.4269
Epoch 2/100
70/70 [==============================] - 0s 3ms/step - loss: 1.3844 - accuracy: 0.6763 - val_loss: 0.9555 - val_accuracy: 0.7018
Epoch 3/100
70/70 [==============================] - 0s 3ms/step - loss: 0.6244 - accuracy: 0.8035 - val_loss: 0.6510 - val_accuracy: 0.7193
Epoch 4/100
70/70 [==============================] - 0s 3ms/step - loss: 0.5966 - accuracy: 0.8092 - val_loss: 1.1851 - val_accuracy: 0.7602
Epoch 5/100
70/70 [==============================] - 0s 3ms/step - loss: 0.4247 - accuracy: 0.8584 - val_loss: 1.0088 - val_accuracy: 0.7895
Epoch 6/100
70/70 [==============================] - 0s 3ms/step - loss: 0.4635 - accuracy: 0.8497 - val_loss: 0.3302 - val_accuracy: 0.8480
Epoch 7/100
70/70 [==============================] - 0s 3ms/step - loss: 0.3803 - accuracy: 0.8844 - val_loss: 0.3583 - val_accuracy: 0.8129
Epoch 8/100
7

In [17]:
scores=model.evaluate(x,y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

17/17 [==============================] - 0s 1ms/step - loss: 0.9031 - accuracy: 0.8162
accuracy: 81.62%
